<a href="https://colab.research.google.com/github/CarlosMejia07/Proyecto-Kaggle-UdeA-20252/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importe, descompresión e inspección de los datos**

In [27]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:    296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


**Importe de las librerias a usar y lectura del archivo train**

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

z = pd.read_csv("train.csv")
print ("Estructura de los datos cargados", z.shape)
z_clean = z.copy()

Estructura de los datos cargados (692500, 21)


**Se dejan de lado las columnas que no aporten a la predicción, preliminarmente**

In [46]:
z_clean = z_clean.drop(columns=['ID', 'PERIODO_ACADEMICO'])

**Detección y reemplazo de valores nulos**

In [49]:
# Separar valores numéricos y no numéricos
numeric_cols = z_clean.select_dtypes(include=np.number).columns
non_numeric_cols = z_clean.select_dtypes(exclude=np.number).columns

# Reemplazar valores nulos de columnas numéricas con la media
z_clean[numeric_cols] = z_clean[numeric_cols].fillna(z_clean[numeric_cols].median())

# Reemplazar valores nulos de columnas no numéricas por la moda
for col in non_numeric_cols:
    z_clean[col] = z_clean[col].fillna(z_clean[col].mode()[0])

z_clean.isna().sum()

,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_VALORMATRICULAUNIVERSIDAD,0
E_HORASSEMANATRABAJA,0
F_ESTRATOVIVIENDA,0
F_TIENEINTERNET,0
F_EDUCACIONPADRE,0
F_TIENELAVADORA,0
F_TIENEAUTOMOVIL,0
E_PRIVADO_LIBERTAD,0


**Codificación de las variables con valores nominales y ordinales**

In [50]:
# Reemplazar valores si & no por 0 & 1
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
z_clean.replace({'Si': 1, 'No': 0, 'N': 0}, inplace=True)

# Reemplazar valores de variables ordinales
z_clean['F_EDUCACIONPADRE'] = z_clean['F_EDUCACIONPADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')
z_clean['F_EDUCACIONMADRE'] = z_clean['F_EDUCACIONMADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')

orden_educacion = [
    'Desconocido',
    'Ninguno',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta',
    'Educación profesional completa',
    'Postgrado'
]
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[orden_educacion])
z_clean['F_EDUCACIONPADRE'] = encoder.fit_transform(z_clean[['F_EDUCACIONPADRE']])
z_clean['F_EDUCACIONMADRE'] = encoder.fit_transform(z_clean[['F_EDUCACIONMADRE']])


In [51]:
orden_matrícula = [
    'No pagó matrícula',
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones'
]
encoder = OrdinalEncoder(categories=[orden_matrícula])
z_clean['E_VALORMATRICULAUNIVERSIDAD'] = encoder.fit_transform(z_clean[['E_VALORMATRICULAUNIVERSIDAD']])

In [52]:
orden_estrato = [
    'Sin Estrato',
    'Estrato 1',
    'Estrato 2',
    'Estrato 3',
    'Estrato 4',
    'Estrato 5',
    'Estrato 6'
]
encoder = OrdinalEncoder(categories=[orden_estrato])
z_clean['F_ESTRATOVIVIENDA'] = encoder.fit_transform(z_clean[['F_ESTRATOVIVIENDA']])

In [53]:
orden_trabajo = [
    '0',
    'Menos de 10 horas',
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',
    'Más de 30 horas'
]
encoder = OrdinalEncoder(categories=[orden_trabajo])
z_clean['E_HORASSEMANATRABAJA'] = encoder.fit_transform(z_clean[['E_HORASSEMANATRABAJA']])

**Ahora las variables nominales (no tienen orden específico)**

In [54]:
# Se realiza un One-Hot Encoding
z_clean = pd.get_dummies(z_clean, columns=['E_PRGM_DEPARTAMENTO', 'E_PRGM_ACADEMICO'])

**Estandarizado de las variables ordinales, necesario o no según el o los modelos que se usarán**

In [55]:
# Con media = 0, desviación estándar = 1
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_ordinales = ['E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA',
                  'E_VALORMATRICULAUNIVERSIDAD', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']

z_clean[cols_ordinales] = scaler.fit_transform(z_clean[cols_ordinales])

In [56]:
# Variables numéricas
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

z_clean[cols_numericas] = scaler.fit_transform(z_clean[cols_numericas])

**Variable objetivo**

In [48]:
# Definimos el orden lógico
orden = ['bajo', 'medio-bajo', 'medio-alto', 'alto']

z_clean['RENDIMIENTO_GLOBAL'] = pd.Categorical(
    z_clean['RENDIMIENTO_GLOBAL'],
    categories=orden,
    ordered=True
).codes

In [59]:
# Visualización de los datos
z_clean.head(10)

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,E_PRGM_ACADEMICO_TEOLOGÍA,E_PRGM_ACADEMICO_TERAPIA CARDIORRESPIRATORIA,E_PRGM_ACADEMICO_TERAPIA OCUPACIONAL,E_PRGM_ACADEMICO_TERAPIA RESPIRATORIA,E_PRGM_ACADEMICO_TERAPIAS PSICOSOCIALES,E_PRGM_ACADEMICO_TRABAJO SOCIAL,E_PRGM_ACADEMICO_TRADUCCION INGLES-FRANCES-ESPAÑOL,E_PRGM_ACADEMICO_TURISMO,E_PRGM_ACADEMICO_URBANISMO,E_PRGM_ACADEMICO_ZOOTECNIA
0,1.383918,-0.971956,0.456215,1,0.356101,1,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
1,0.270116,-1.628413,0.456215,0,0.714988,1,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
2,0.270116,0.997418,0.456215,1,-0.002786,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,0.827017,-1.628413,1.363766,1,-1.797221,1,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
4,0.270116,0.340960,0.456215,1,-0.720560,1,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
5,1.940819,-0.971956,2.271316,1,1.432762,1,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
6,0.270116,0.340960,-0.451335,1,1.073875,1,1,0,1,1,...,False,False,False,False,False,False,False,False,False,False
7,-0.286785,-0.315498,-0.451335,1,-1.079447,1,0,0,1,0,...,False,False,False,False,False,False,False,False,False,False
8,1.383918,-0.971956,-1.358885,1,-0.002786,1,1,0,1,1,...,False,False,False,False,False,False,False,False,False,False
9,0.270116,0.997418,2.271316,1,1.791650,1,1,0,1,1,...,False,False,False,False,False,False,False,False,False,False


**Variable predictoria y objetivo**

In [60]:
X = z_clean.drop(columns=['RENDIMIENTO_GLOBAL'])
y = z_clean['RENDIMIENTO_GLOBAL']

,RENDIMIENTO_GLOBAL
0,2
1,0
2,0
3,3
4,1
...,...
692495,2
692496,0
692497,1
692498,0
